In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
usecols = ['register_type', 'device_type', 'user_reg_days', 'user_lastweek_launchday', 'user_last_launch_dist', 'user_hist_launch_freq', 'user_hist_launchday', 'user_mean_continue_launch_times_lastweek', 'user_max_continue_launch_times_lastweek', 'user_activity_days_hist', 'user_activity_days_lastweek', 'user_min_continue_launch_times_lastweek', 'user_mean_continue_launch_days_lastweek', 'user_max_continue_launch_days_lastweek', 'user_mean_continue_activity_days_lastweek', 'user_max_continue_activity_days_lastweek', 'user_lastweek_act_0_freq', 'user_lastweek_actcount', 'user_lastweek_act_video_uniquecount', 'user_max_continue_launch_times_hist', 'user_min_continue_launch_days_lastweek', 'user_mean_continue_launch_times_hist', 'user_min_continue_activity_days_lastweek', 'user_hist_act_0_count', 'user_hist_actcount', 'user_mean_continue_activity_days_hist', 'user_max_continue_launch_days_hist', 'user_hist_act_freq', 'user_mean_continue_launch_days_hist', 'user_hist_act_author_count', 'user_mean_no_launch_days_hist', 'user_min_activity_daytimes_lastweek', 'user_lastweek_act', 'user_lastweek_act_0', 'user_mean_continue_activity_times_hist', 'user_max_launch_daytimes_lastweek', 'user_lastweek_launch', 'user_lastweek_act_page_3_count', 'user_lastweek_act_page_1_count', 'user_max_no_launch_days_hist', 'user_last_act_date', 'user_lastweek_act_2_freq', 'user_lastweek_video_freq', 'user_lastweek_act_2_count', 'user_lastweek_launch_freq', 'user_max_no_activity_days_lastweek_hist_dist', 'user_lastweek_act_page_2_count', 'user_var_continue_activity_times_lastweek', 'user_kurt_continue_activity_days_hist', 'user_launch_range_percent', 'user_activity_div_launch_days_hist', 'user_hist_act_video_meancount', 'user_hist_video_activity_types', 'user_activity_range_percent', 'user_5daybefore_act_page_1_count', 'user_min_continue_activity_times_5daywin', 'user_lastweek_hist_act_3_count_dist', 'user_lastweek_hist_act_page_3_count_dist', 'user_mean_createvideo_date_lastweek_hist_dist', 'user_4daybefore_act_page_4_count', 'user_kurt_no_launch_days_6daywin', 'user_max_continue_createvideo_days_4daywin', 'user_2daybefore_act_1_count', 'user_max_continue_createvideo_days_hist']
by = ['user_id', 'data_weeknum']
target = ['label']
len(usecols)

64

In [8]:
df = pd.read_csv('./features/b/baseline_features12_ab.csv', usecols=by + target + usecols)

df.shape
df.head()

(198057, 67)

,user_id,data_weeknum,label,register_type,device_type,user_reg_days,user_2daybefore_act_1_count,user_4daybefore_act_page_4_count,user_5daybefore_act_page_1_count,user_activity_days_hist,user_activity_days_lastweek,user_hist_act_0_count,user_hist_act_author_count,user_hist_act_video_meancount,user_hist_actcount,user_hist_launchday,user_hist_video_activity_types,user_kurt_continue_activity_days_hist,user_kurt_no_launch_days_6daywin,user_last_act_date,user_last_launch_dist,user_lastweek_act,user_lastweek_act_0,user_lastweek_act_2_count,user_lastweek_act_page_1_count,user_lastweek_act_page_2_count,user_lastweek_act_page_3_count,user_lastweek_act_video_uniquecount,user_lastweek_actcount,user_lastweek_launch,user_lastweek_launchday,user_max_continue_activity_days_lastweek,user_max_continue_createvideo_days_4daywin,user_max_continue_createvideo_days_hist,user_max_continue_launch_days_hist,user_max_continue_launch_days_lastweek,user_max_continue_launch_times_hist,user_max_continue_launch_times_lastweek,user_max_launch_daytimes_lastweek,user_max_no_launch_days_hist,user_mean_continue_activity_days_hist,user_mean_continue_activity_days_lastweek,user_mean_continue_activity_times_hist,user_mean_continue_launch_days_hist,user_mean_continue_launch_days_lastweek,user_mean_continue_launch_times_hist,user_mean_continue_launch_times_lastweek,user_mean_no_launch_days_hist,user_min_activity_daytimes_lastweek,user_min_continue_activity_days_lastweek,user_min_continue_activity_times_5daywin,user_min_continue_launch_days_lastweek,user_min_continue_launch_times_lastweek,user_var_continue_activity_times_lastweek,user_hist_launch_freq,user_lastweek_launch_freq,user_lastweek_video_freq,user_hist_act_freq,user_lastweek_act_0_freq,user_lastweek_act_2_freq,user_activity_div_launch_days_hist,user_lastweek_hist_act_3_count_dist,user_lastweek_hist_act_page_3_count_dist,user_max_no_activity_days_lastweek_hist_dist,user_mean_createvideo_date_lastweek_hist_dist,user_launch_range_percent,user_activity_range_percent
0,744025_a,0,0,1,283,9,0.0,0.0,0.0,3.0,2.0,109.0,97.0,1.121212,111.0,3,0,-1.0,-1.0,8.0,2,1,1,0.0,0.0,0.0,2.0,68.0,71.0,1,2.0,2.0,0.0,0.0,2,2.0,2,2.0,1.0,5,1.5,2.0,55.5,1.5,2.0,1.5,2.0,2.500000,5.0,2.0,71.0,2.0,2.0,0.0,0.333333,0.285714,0.0,12.333333,10.000000,0.000000,1.000000,0.0,0.0,-5.0,0.0,0.777778,0.777778
1,1270299_a,0,0,1,259,9,0.0,0.0,0.0,2.0,0.0,81.0,18.0,1.370968,85.0,2,0,-1.0,0.0,2.0,8,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,2,0.0,2,0.0,0.0,0,2.0,0.0,85.0,2.0,0.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.222222,0.000000,0.0,9.444444,0.000000,0.000000,1.000000,-4.0,-18.0,0.0,0.0,0.111111,0.111111
2,571220_a,0,0,1,2,9,0.0,0.0,0.0,2.0,0.0,70.0,45.0,1.163934,71.0,2,0,-1.0,0.0,2.0,8,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,2,0.0,2,0.0,0.0,0,2.0,0.0,71.0,2.0,0.0,2.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.222222,0.000000,0.0,7.888889,0.000000,0.000000,1.000000,0.0,-16.0,0.0,0.0,0.111111,0.111111
3,1308501_a,0,1,0,23,9,0.0,0.0,14.0,4.0,2.0,122.0,9.0,4.548387,141.0,4,0,-1.0,-1.0,5.0,5,1,1,0.0,25.0,0.0,34.0,22.0,60.0,1,2.0,1.0,0.0,0.0,3,1.0,3,1.0,1.0,1,2.0,1.0,70.5,2.0,1.0,2.0,1.0,0.333333,14.0,1.0,14.0,1.0,1.0,256.0,0.444444,0.285714,0.0,15.666667,5.857143,0.000000,1.000000,0.0,-35.0,0.0,0.0,0.444444,0.444444
4,745554_a,0,1,2,0,9,0.0,0.0,0.0,5.0,3.0,29.0,21.0,1.476190,31.0,6,0,-1.0,-1.0,7.0,3,1,1,1.0,0.0,18.0,0.0,17.0,26.0,1,4.0,3.0,0.0,0.0,4,4.0,4,4.0,1.0,1,2.5,3.0,15.5,3.0,4.0,3.0,4.0,0.200000,3.0,3.0,26.0,4.0,4.0,0.0,0.666667,0.571429,0.0,3.444444,3.428571,0.142857,0.833333,0.0,0.0,-2.0,0.0,0.666667,0.666667


In [6]:
# df = df[df.label != -1]
# df = df[by + target + usecols]

In [17]:
df['user_id'] = df.user_id.astype(str)

train = df[(df.data_weeknum < df.data_weeknum.max()-1)]
val = df[(df.data_weeknum == df.data_weeknum.max()-1) & (df.user_id.map(lambda x:'_a' not in x))]
test = df[df.data_weeknum == df.data_weeknum.max()]
trainval = df[df.data_weeknum <df.data_weeknum.max()]

## XGB

In [20]:
import xgboost as xgb
from sklearn.metrics import f1_score, recall_score, precision_score

In [6]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    f1s = []
    for limit in np.arange(0.4, 0.44, 0.01):
        pred = [int(i>limit) for i in preds]
        f1s.append(f1_score(labels, pred))
#     pred = [1. if i>0.45 else 0. for i in preds]
    # return a pair metric_name, result
    # since preds are margin(before logistic transformation, cutoff at 0)
    return 'f1', -1*max(f1s) #-1*f1_score(labels, pred)

In [18]:
# ab
params = {
        'objective': 'binary:logistic',
        'eta': 0.05,
#         'colsample_bytree': 0.3,
#         'subsample':0.9,
        'min_child_weight': 17,
        'max_depth': 5,
        'gamma':0.1,
        'seed': 2018,
        'missing': -1,
        'n_jobs':-1,
        'eval_metric':'auc',
#         'tree_method':'gpu_hist',
#         'max_bin':64,
#         'gpu_device':0
}
# f1 0.808476
dtrain = xgb.DMatrix(train[usecols], label=train[['label']])
dval = xgb.DMatrix(val[usecols], label=val[['label']])
model = xgb.train(params, dtrain, num_boost_round=20000, early_stopping_rounds=50, evals=[(dtrain, 'train'),(dval, 'val')],feval=evalerror,verbose_eval=100)

[0]	train-auc:0.868453	val-auc:0.882834	train-f1:-0.694072	val-f1:-0.656362
Multiple eval metrics have been passed: 'val-f1' will be used for early stopping.

Will train until val-f1 hasn't improved in 50 rounds.
[100]	train-auc:0.880169	val-auc:0.890111	train-f1:-0.812487	val-f1:-0.810449
Stopping. Best iteration:
[107]	train-auc:0.880463	val-auc:0.890169	train-f1:-0.812709	val-f1:-0.810907



In [7]:
# only b
params = {
        'objective': 'binary:logistic',
        'eta': 0.05,
#         'colsample_bytree': 0.3,
#         'subsample':0.9,
        'min_child_weight': 17,
        'max_depth': 5,
        'gamma':0.1,
        'seed': 2018,
        'missing': -1,
        'n_jobs':-1,
        'eval_metric':'auc',
#         'tree_method':'gpu_hist',
#         'max_bin':64,
#         'gpu_device':0
}
# f1 0.808476
dtrain = xgb.DMatrix(train[usecols], label=train[['label']])
dval = xgb.DMatrix(val[usecols], label=val[['label']])
model = xgb.train(params, dtrain, num_boost_round=20000, early_stopping_rounds=50, evals=[(dtrain, 'train'),(dval, 'val')],feval=evalerror,verbose_eval=100)

[0]	train-auc:0.868518	val-auc:0.879792	train-f1:-0.698054	val-f1:-0.656362
Multiple eval metrics have been passed: 'val-f1' will be used for early stopping.

Will train until val-f1 hasn't improved in 50 rounds.
[100]	train-auc:0.881919	val-auc:0.889909	train-f1:-0.815661	val-f1:-0.809165
Stopping. Best iteration:
[149]	train-auc:0.884027	val-auc:0.889694	train-f1:-0.817099	val-f1:-0.809664



In [21]:
pred = model.predict(dval, ntree_limit=model.best_ntree_limit)
limits = []
for limit in np.arange(0.4, 0.46, 0.01):
    preds = [int(i>limit) for i in pred]
    score = f1_score(dval.get_label(), preds)
    limits.append((limit, -1*score))
    print(limit, score, precision_score(dval.get_label(), preds), recall_score(dval.get_label(), preds))#0.807701253391911 0.763335287221571 0.8575426658618229
limits.sort(key=lambda x:x[1])
best_limit = round(limits[0][0],2)
print('best_limit:', best_limit, 'score:', limits[0][1])

0.4 0.8091885502852151 0.7614119922630561 0.8633622107687247
0.41000000000000003 0.8096801716782585 0.7660844464014873 0.8585371202982783
0.42000000000000004 0.8101015360583183 0.7712671029149316 0.8530540629454985
0.43000000000000005 0.810907090656678 0.7763177691960479 0.8487224476368023
0.44000000000000006 0.8113425925925926 0.7797552836484983 0.8455971049457177
0.45000000000000007 0.8117827489555239 0.7839632277834525 0.8416493036517162
best_limit: 0.45 score: -0.8117827489555239


In [22]:
dtrainval = xgb.DMatrix(trainval[usecols], label=trainval[['label']])
dtest = xgb.DMatrix(test[usecols], label=test[['label']])
model = xgb.train(params, dtrainval, num_boost_round=model.best_iteration + 1)

In [25]:
test['pred'] = model.predict(dtest)
save = test[test.pred>0.41].sort_values('pred', ascending=False)[['user_id']]
print(save.shape)

(24163, 1)


In [27]:
save.to_csv('./result/b/xgb_baseline9_0627_f1_limit041.csv',index=False, header=None)

## Feature Select

In [28]:
from model.MLFeatureSelection import sequence_selection, importance_selection, coherence_selection, tools
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score

In [29]:
def score(labels, preds):
    f1s = []
    for limit in np.arange(0.4, 0.44, 0.01):
        pred = [int(i > limit) for i in preds]
        f1s.append(f1_score(labels, pred))
    return max(f1s)


def validate(X, y, features, clf, lossfunction):
    totaltest = 0
    for day in [2]:
        T = (X.data_weeknum != day)
        X_train, X_test = X[T], X[~T]
        X_train, X_test = X_train[features], X_test[features]
        y_train, y_test = y[T], y[~T]
        clf.fit(X_train,y_train, eval_set = [(X_train, y_train), (X_test, y_test)], eval_metric='auc', verbose=False, early_stopping_rounds=20)
#         print(lossfunction(y_test, clf.predict_proba(X_test)[:, 1]))
        totaltest += lossfunction(y_test, clf.predict_proba(X_test)[:, 1])
#         print(totaltest)
    totaltest /= 1.0
    return totaltest, clf

In [30]:
def add(x,y):
    return x + y

def substract(x,y):
    return x - y

def times(x,y):
    return x * y

def divide(x,y):
    return (x + 0.001)/(y + 0.001)

CrossMethod = {'+':add,
               '-':substract,
               '*':times,
               '/':divide,}

In [32]:
notusable = by + target
f = usecols

sf = sequence_selection.Select(Sequence=True, Random=True, Cross=True)  # 初始化选择器，选择你需要的流程
sf.ImportDF(trainval, label='label')  # 导入数据集以及目标标签
sf.ImportLossFunction(score, direction='ascend')  # 导入评价函数以及优化方向
sf.ImportCrossMethod(CrossMethod)
sf.InitialNonTrainableFeatures(notusable)  # 初始化不能用的特征
sf.InitialFeatures(usecols)
sf.GenerateCol()  # 生成特征库
# sf.SetSample(0.1, samplemode = 0, samplestate = 0)
sf.clf = xgb.XGBClassifier(
    objective='binary:logistic',
    learning_rate=0.05,
    n_estimators=5000,
    min_child_weight=17, 
    max_depth=5,
    gamma=0.1,
    seed=2018,
    missing=-1,
    n_jobs=10)
sf.SetLogFile('./log/record_seq_b.log')  # 初始化日志文件
sf.run(validate)  # 输入检验函数并开始运行

Features Quantity Limit: inf
Time Limit: inf min(s)
test performance of initial features combination
Mean loss: 0.8086764241339371
--------------------start greedy--------------------
register_type
device_type
user_reg_days
user_lastweek_launchday
user_last_launch_dist
user_hist_launch_freq
user_hist_launchday
user_mean_continue_launch_times_lastweek
user_max_continue_launch_times_lastweek
user_activity_days_hist
user_activity_days_lastweek
user_min_continue_launch_times_lastweek
user_mean_continue_launch_days_lastweek
user_max_continue_launch_days_lastweek
user_mean_continue_activity_days_lastweek
user_max_continue_activity_days_lastweek
user_lastweek_act_0_freq
user_lastweek_actcount
user_lastweek_act_video_uniquecount
user_max_continue_launch_times_hist
user_min_continue_launch_days_lastweek
user_mean_continue_launch_times_hist
user_min_continue_activity_days_lastweek
user_hist_act_0_count
user_hist_actcount
user_mean_continue_activity_days_hist
user_max_continue_launch_days_hist
us

['register_type',
 'device_type',
 'user_reg_days',
 'user_lastweek_launchday',
 'user_last_launch_dist',
 'user_hist_launchday',
 'user_max_continue_launch_times_lastweek',
 'user_activity_days_lastweek',
 'user_min_continue_launch_times_lastweek',
 'user_mean_continue_launch_days_lastweek',
 'user_max_continue_launch_days_lastweek',
 'user_lastweek_actcount',
 'user_lastweek_act_video_uniquecount',
 'user_max_continue_launch_times_hist',
 'user_min_continue_launch_days_lastweek',
 'user_hist_act_0_count',
 'user_hist_actcount',
 'user_mean_continue_activity_days_hist',
 'user_max_continue_launch_days_hist',
 'user_hist_act_freq',
 'user_mean_continue_launch_days_hist',
 'user_hist_act_author_count',
 'user_mean_no_launch_days_hist',
 'user_min_activity_daytimes_lastweek',
 'user_lastweek_act',
 'user_lastweek_act_0',
 'user_mean_continue_activity_times_hist',
 'user_max_launch_daytimes_lastweek',
 'user_lastweek_launch',
 'user_lastweek_act_page_3_count',
 'user_lastweek_act_page_1_c

In [12]:
cols = ['register_type',
 'device_type',
 'user_reg_days',
 'user_lastweek_launchday',
 'user_last_launch_dist',
 'user_max_continue_launch_times_lastweek',
 'user_activity_days_hist',
 'user_activity_days_lastweek',
 'user_mean_continue_launch_days_lastweek',
 'user_max_continue_launch_days_lastweek',
 'user_mean_continue_activity_days_lastweek',
 'user_max_continue_activity_days_lastweek',
 'user_lastweek_actcount',
 'user_lastweek_act_video_uniquecount',
 'user_max_continue_launch_times_hist',
 'user_min_continue_launch_days_lastweek',
 'user_mean_continue_launch_times_hist',
 'user_min_continue_activity_days_lastweek',
 'user_hist_actcount',
 'user_mean_continue_activity_days_hist',
 'user_max_continue_launch_days_hist',
 'user_hist_act_freq',
 'user_mean_continue_launch_days_hist',
 'user_hist_act_author_count',
 'user_mean_no_launch_days_hist',
 'user_min_activity_daytimes_lastweek',
 'user_lastweek_act',
 'user_lastweek_act_0',
 'user_mean_continue_activity_times_hist',
 'user_max_launch_daytimes_lastweek',
 'user_lastweek_launch',
 'user_lastweek_act_page_3_count',
 'user_lastweek_act_page_1_count',
 'user_max_no_launch_days_hist',
 'user_last_act_date',
 'user_lastweek_act_2_freq',
 'user_lastweek_video_freq',
 'user_lastweek_act_2_count',
 'user_lastweek_launch_freq',
 'user_max_no_activity_days_lastweek_hist_dist',
 'user_lastweek_act_page_2_count',
 'user_var_continue_activity_times_lastweek',
 'user_kurt_continue_activity_days_hist',
 'user_launch_range_percent',
 'user_activity_div_launch_days_hist',
 'user_activity_range_percent',
 'user_5daybefore_act_page_1_count',
 'user_min_continue_activity_times_5daywin',
 'user_lastweek_hist_act_3_count_dist',
 'user_4daybefore_act_page_4_count',
 'user_kurt_no_launch_days_6daywin',
 'user_max_continue_createvideo_days_4daywin',
 'user_2daybefore_act_1_count',
 'user_max_continue_createvideo_days_hist',
 '(device_type-user_activity_days_lastweek)',
 '(user_lastweek_launchday-user_min_continue_launch_days_lastweek)',
 '(user_last_launch_dist/user_lastweek_act_video_uniquecount)',
 '(user_activity_days_hist-user_kurt_continue_activity_days_hist)',
 '(user_activity_days_hist/user_var_continue_activity_times_lastweek)',
 '(user_activity_days_lastweek-user_max_continue_launch_times_hist)',
 '(user_activity_days_lastweek/user_max_continue_activity_days_lastweek)',
 '(user_lastweek_actcount-user_activity_days_lastweek)',
 '(user_lastweek_act_video_uniquecount-user_max_continue_activity_days_lastweek)',
 '(user_min_continue_launch_days_lastweek*user_2daybefore_act_1_count)',
 '(user_hist_act_author_count*user_max_continue_createvideo_days_4daywin)',
 '(user_max_launch_daytimes_lastweek*user_5daybefore_act_page_1_count)',
 '(user_max_launch_daytimes_lastweek*user_lastweek_act_page_3_count)']
len(cols)

67

In [25]:
cols = ['register_type',
 'device_type',
 'user_reg_days',
 'user_last_launch_dist',
 'user_hist_launch_freq',
 'user_hist_launchday',
 'user_max_continue_launch_times_lastweek',
 'user_activity_days_hist',
 'user_activity_days_lastweek',
 'user_min_continue_launch_times_lastweek',
 'user_mean_continue_launch_days_lastweek',
 'user_max_continue_launch_days_lastweek',
 'user_mean_continue_activity_days_lastweek',
 'user_max_continue_activity_days_lastweek',
 'user_lastweek_act_0_freq',
 'user_lastweek_actcount',
 'user_lastweek_act_video_uniquecount',
 'user_min_continue_launch_days_lastweek',
 'user_mean_continue_launch_times_hist',
 'user_min_continue_activity_days_lastweek',
 'user_hist_actcount',
 'user_mean_continue_activity_days_hist',
 'user_max_continue_launch_days_hist',
 'user_hist_act_freq',
 'user_mean_continue_launch_days_hist',
 'user_hist_act_author_count',
 'user_mean_no_launch_days_hist',
 'user_min_activity_daytimes_lastweek',
 'user_lastweek_act',
 'user_lastweek_act_0',
 'user_mean_continue_activity_times_hist',
 'user_max_launch_daytimes_lastweek',
 'user_lastweek_launch',
 'user_lastweek_act_page_3_count',
 'user_lastweek_act_page_1_count',
 'user_max_no_launch_days_hist',
 'user_last_act_date',
 'user_lastweek_act_2_freq',
 'user_lastweek_video_freq',
 'user_lastweek_act_2_count',
 'user_lastweek_launch_freq',
 'user_max_no_activity_days_lastweek_hist_dist',
 'user_lastweek_act_page_2_count',
 'user_var_continue_activity_times_lastweek',
 'user_kurt_continue_activity_days_hist',
 'user_launch_range_percent',
 'user_activity_div_launch_days_hist',
 'user_hist_act_video_meancount',
 'user_hist_video_activity_types',
 'user_activity_range_percent',
 'user_5daybefore_act_page_1_count',
 'user_min_continue_activity_times_5daywin',
 'user_lastweek_hist_act_3_count_dist',
 'user_lastweek_hist_act_page_3_count_dist',
 'user_mean_createvideo_date_lastweek_hist_dist',
 'user_4daybefore_act_page_4_count',
 'user_kurt_no_launch_days_6daywin',
 'user_max_continue_createvideo_days_4daywin',
 'user_2daybefore_act_1_count',
 'user_max_continue_createvideo_days_hist',
 '(register_type+user_lastweek_video_freq)',
 '(register_type*user_hist_launchday)',
 '(user_hist_launchday/user_2daybefore_act_1_count)',
 '(user_activity_days_lastweek-user_mean_continue_activity_days_lastweek)',
 '(user_min_continue_launch_times_lastweek*user_lastweek_hist_act_3_count_dist)',
 '(user_mean_continue_activity_days_lastweek+user_max_no_activity_days_lastweek_hist_dist)',
 '(user_lastweek_act_0_freq-user_4daybefore_act_page_4_count)',
 '(user_max_continue_launch_days_hist*user_5daybefore_act_page_1_count)',
 '(user_var_continue_activity_times_lastweek*user_5daybefore_act_page_1_count)',
 '(user_launch_range_percent/user_4daybefore_act_page_4_count)',
 '(user_launch_range_percent*user_4daybefore_act_page_4_count)',
 '(user_5daybefore_act_page_1_count/user_lastweek_launch)',
 '(user_5daybefore_act_page_1_count*user_min_continue_launch_days_lastweek)',
 '(user_4daybefore_act_page_4_count/user_max_launch_daytimes_lastweek)']
len(cols)

74